##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Сегментация изображений

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/segmentation">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/segmentation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/segmentation.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/images/segmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве основное внимание уделяется задаче сегментации изображений с использованием модифицированного <a href="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/" class="external"> U-Net </a>.

## Что такое сегментация?
До сих пор вы видели классификацию изображений, в которой задача сети - присвоить метку или класс входному изображению. Однако предположим, что вы хотите знать, где находится объект на изображении, форму этого объекта, какой пиксель принадлежит какому объекту и т.д. В этом случае вам нужно сегментировать изображение, т.е. каждый пиксель изображения должен получить ярлык. Таким образом, задача сегментации изображения - научить нейронную сеть выводить пиксельную маску изображения. Это помогает понять изображение на гораздо более низком уровне, то есть на уровне пикселей. Сегментация изображений применяется в медицинской визуализации, беспилотных автомобилях и спутниковой съемке, и это лишь некоторые из них.

Набор данных, который будет использоваться в этом руководстве, - это [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/), созданный Parkhi *et al*. Набор данных состоит из изображений, соответствующих им меток и пиксельных масок. Маски - это метки для каждого пикселя. Каждому пикселю дается одна из трех категорий:

* Класс 1: пиксель, принадлежащий питомцу.
* Класс 2: пиксель, граничащий с животным.
* Класс 3: Ничего из вышеперечисленного/Окружающий пиксель.

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Загрузка датасета Oxford-IIIT Pets

Oxford-IIIT Pets уже включен в наборы данных TensorFlow, все, что нужно сделать, это загрузить его. Маски сегментации включены в версию 3+..

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Следующий код выполняет простую аугментацию, переворачивания изображения. Кроме того, изображение нормализуется до [0,1]. Наконец, как упоминалось выше, пиксели в маске сегментации помечены как {1, 2, 3}. Для удобства вычтем 1 из маски сегментации, в результате чего получатся следующие метки: {0, 1, 2}.

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

Набор данных уже содержит необходимое разделение на тренировочную и проверочную выборки, поэтому давайте будем использовать то же разделение.

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [ ]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Давайте посмотрим на пример изображения из датасета и соответствующую ему маску.

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## Определение модели
Используемая здесь модель представляет собой модифицированную U-Net. U-Net состоит из кодировщика (понижающего дискретизатора) и декодера (повышающего дискретизатора). Для того, чтобы модель использовала сильные функции и для уменьшения количества обучаемых параметров, в качестве кодировщика можно использовать предварительно обученную модель. Поэтому кодировщиком для этой задачи будет предварительно обученная модель MobileNetV2, чьи промежуточные выходы будут использоваться, а декодером будет блок повышающей дискретизации, уже реализованный в примерах TensorFlow в [учебнике Pix2pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). 

The reason to output three channels is because there are three possible labels for each pixel. Think of this as multi-classification where each pixel is being classified into three classes.
Причина вывода трех каналов на пиксель заключается в том, что для каждого пикселя есть три возможных метки. Подумайте об этом как о мультиклассификации, когда каждый пиксель классифицируется на три класса.

In [ ]:
OUTPUT_CHANNELS = 3

Как уже упоминалось, кодировщиком будет предварительно обученная модель MobileNetV2, которая готова к использованию в [tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras /Приложения). Кодировщик состоит из определенных выходных данных из промежуточных слоев модели. Учтите, что кодировщик не будет обучаться в процессе обучения.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Используйем активации этих слоев
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Создаем модель извлечения признаков
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

Декодер/повышающий дискретизатор - это просто серия блоков повышающей дискретизации, реализованных в примерах TensorFlow.

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Увеличение размерности данных
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Уменьшение размерности данных и установление пропуска соединений
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # Последний слой модели
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

## Обучение модели

Теперь осталось скомпилировать и обучить модель. Используемая здесь функция потерь - это `losses.SparseCategoricalCrossentropy(from_logits=True)`. Причина использования этой функции потерь заключается в том, что сеть пытается присвоить каждому пикселю метку, точно так же, как при прогнозировании нескольких классов. В истинной маске сегментации каждый пиксель имеет одно значение из {0,1,2}. Здесь сеть выводит три канала. По сути, каждый канал пытается научиться предсказывать класс, и `losses.SparseCategoricalCrossentropy(from_logits=True)` являются рекомендуемыми потерями для такого сценария. Используя выходной сигнал сети, метка, присвоенная пикселю, представляет собой канал с наибольшим значением. Это то, что делает функция create_mask.

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Взглянем на получившуюся архитектуру модели:

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

Давайте посмотрим перед обучением, что предсказывает модель.

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

Давайте посмотрим, как модель улучшается в процессе обучения. Для выполнения этой задачи ниже определена функция обратного вызова.

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Прогнозирование

Сделаем некоторые прогнозы. В целях экономии времени количество эпох оставлено небольшим, но вы можете установить его больше, чтобы получить более точные результаты.

In [ ]:
show_predictions(test_dataset, 3)

## Что дальше?

Теперь, когда вы понимаете, что такое сегментация изображений и как она работает, вы можете попробовать заново с различными выходными данными промежуточного слоя или даже с другой предварительно обученной моделью. Вы также можете испытать себя, попробовав задачу по сегментации изображений [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge/overview), размещенную на Kaggle.

Также можете изучить [API обнаружения объектов Tensorflow](https://github.com/tensorflow/models/tree/master/research/object_detection) для другой модели, которую вы можете переобучить на собственных данных.